<a href="https://colab.research.google.com/github/YianKim/2020_EDAProject/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 불러오기


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
train_data = data[range(10000)].reshape([10000,28,28,1])
train_labels = labels[range(10000)].reshape([10000,1])

In [ ]:
unlab_data = data[range(10000,60000)].reshape([50000,28,28,1])
unlab_labels = labels[range(10000,60000)].reshape([50000,1])

In [ ]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

In [ ]:
train_labels2 = []
unlab_labels2 = []
test_labels2 = []

In [ ]:
for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[train_labels[i][0]] = 1
  train_labels2.append(white)

for i in range(50000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[unlab_labels[i][0]] = 1
  unlab_labels2.append(white)

for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[test_labels[i][0]] = 1
  test_labels2.append(white)

In [ ]:
train_labels = np.array(train_labels2)
unlab_labels = np.array(unlab_labels2)
test_labels = np.array(test_labels2)

## CNN

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
110/110 [==============================] - 1s 9ms/step - loss: 2.7861 - accuracy: 0.6353 - val_loss: 0.8022 - val_accuracy: 0.7300 - lr: 0.0010
Epoch 2/2000
110/110 [==============================] - 1s 7ms/step - loss: 0.6622 - accuracy: 0.7697 - val_loss: 0.6885 - val_accuracy: 0.7657 - lr: 0.0010
Epoch 3/2000
110/110 [==============================] - 1s 6ms/step - loss: 0.5660 - accuracy: 0.7983 - val_loss: 0.6609 - val_accuracy: 0.7703 - lr: 0.0010
Epoch 4/2000
110/110 [==============================] - 1s 6ms/step - loss: 0.5037 - accuracy: 0.8180 - val_loss: 0.6342 - val_accuracy: 0.7733 - lr: 0.0010
Epoch 5/2000
110/110 [==============================] - 1s 9ms/step - loss: 0.4535 - accuracy: 0.8319 - val_loss: 0.6077 - val_accuracy: 0.7847 - lr: 0.0010
Epoch 6/2000
110/110 [==============================] - 1s 7ms/step - loss: 0.4147 - accuracy: 0.8441 - val_loss: 0.6093 - val_accuracy: 0.7960 - lr: 0.0010
Epoch 7/2000
110/110 [==============================] - 1s

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## 모델 결과

In [ ]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model_eval()

0.8829

# 라벨링

## 라벨링; 1

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [ ]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(temp2/30)
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 50000/50000 [00:07<00:00, 6457.61it/s]


In [ ]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [ ]:
Counter(np.argmax(unlab_labels, axis=1))

Counter({0: 5058,
         1: 4973,
         2: 4984,
         3: 4981,
         4: 5026,
         5: 5011,
         6: 4979,
         7: 4978,
         8: 5010,
         9: 5000})

### 불확정성 컷 1

In [ ]:
# upper bound of variance?

UB = []

for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB.append(np.percentile(classvars, 50))

100%|██████████| 50000/50000 [01:24<00:00, 589.32it/s]


In [ ]:
lowvars = []
ind = 0 

for i in Vars:
  if i <= UB[np.argmax(unlab_labels, axis=1)[ind]]:
    lowvars.append(ind)
  ind += 1

In [ ]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [ ]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), 50000)
randomindx2 = randomindx[0:25000]
randomindx = randomindx[25000:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.89192

In [ ]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 95% 이상
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.9728878433874247

In [ ]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [ ]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [ ]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[randomindx2]
unlab_labels_2 = unlab_labels[randomindx2]

In [ ]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3600,
         1: 3599,
         2: 3613,
         3: 3638,
         4: 3469,
         5: 3491,
         6: 3151,
         7: 3507,
         8: 3505,
         9: 3508})

In [ ]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3409,
         1: 3473,
         2: 3582,
         3: 3546,
         4: 3512,
         5: 3488,
         6: 3448,
         7: 3581,
         8: 3447,
         9: 3514})

In [ ]:
Counter(np.argmax(unlab_labels_1, axis=1))

Counter({0: 2529,
         1: 2407,
         2: 2492,
         3: 2490,
         4: 2513,
         5: 2505,
         6: 2489,
         7: 2489,
         8: 2505,
         9: 2500})

In [ ]:
Counter(np.argmax(unlab_labels_2, axis=1))

Counter({0: 2565,
         1: 2496,
         2: 2452,
         3: 2497,
         4: 2506,
         5: 2458,
         6: 2504,
         7: 2474,
         8: 2554,
         9: 2494})

### 불확정성 컷 2

In [ ]:
# upper bound of variance?

UB25 = []
UB50 = []
UB75 = []


for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB25.append(np.percentile(classvars, 25))
  UB50.append(np.percentile(classvars, 50))
  UB75.append(np.percentile(classvars, 75))

100%|██████████| 50000/50000 [01:20<00:00, 620.73it/s]


In [ ]:
# UB25 < UB50 < UB75

In [ ]:
vars25 = []
vars50 = []
vars75 = []
vars100 = []

ind = 0 


for i in Vars:
  if i <= UB25[np.argmax(unlab_labels, axis=1)[ind]]:
    vars25.append(ind)
  elif i <= UB50[np.argmax(unlab_labels, axis=1)[ind]]:
    vars50.append(ind)
  elif i <= UB75[np.argmax(unlab_labels, axis=1)[ind]]:
    vars75.append(ind)
  else:
    vars100.append(ind)
  ind += 1

In [ ]:
k1 = random.sample(range(len(vars25)), len(vars25))
k2 = random.sample(range(len(vars50)), len(vars50))
k3 = random.sample(range(len(vars75)), len(vars75))
k4 = random.sample(range(len(vars100)), len(vars100))

lowvars = k1[0:np.int(len(k1)/2)] + k2[0:np.int(len(k2)/2)] + k3[0:np.int(len(k3)/2)] + k4[0:np.int(len(k4)/2)]
highvars = k1[np.int(len(k1)/2):len(k1)] + k2[np.int(len(k2)/2):len(k2)] + k3[np.int(len(k3)/2):len(k3)] + k4[np.int(len(k4)/2):len(k4)]

In [ ]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), 50000)
randomindx2 = randomindx[0:25000]
randomindx = randomindx[25000:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.89508

In [ ]:
# 층화추출
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.8944757790311613

In [ ]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [ ]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [ ]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[randomindx2]
unlab_labels_2 = unlab_labels[randomindx2]

In [ ]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3346,
         1: 3485,
         2: 3492,
         3: 3595,
         4: 3516,
         5: 3507,
         6: 3528,
         7: 3500,
         8: 3510,
         9: 3520})

In [ ]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3375,
         1: 3463,
         2: 3521,
         3: 3513,
         4: 3574,
         5: 3426,
         6: 3541,
         7: 3583,
         8: 3469,
         9: 3535})

In [ ]:
Counter(np.argmax(unlab_labels_1, axis=1))

Counter({0: 2542,
         1: 2491,
         2: 2451,
         3: 2505,
         4: 2433,
         5: 2509,
         6: 2676,
         7: 2556,
         8: 2313,
         9: 2525})

In [ ]:
Counter(np.argmax(unlab_labels_2, axis=1))

Counter({0: 2532,
         1: 2500,
         2: 2525,
         3: 2539,
         4: 2434,
         5: 2522,
         6: 2487,
         7: 2463,
         8: 2514,
         9: 2484})

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 5s 12ms/step - loss: 1.1925 - accuracy: 0.6126 - val_loss: 0.6793 - val_accuracy: 0.7716 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.6240 - accuracy: 0.7889 - val_loss: 0.5945 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.5377 - accuracy: 0.8277 - val_loss: 0.5035 - val_accuracy: 0.8390 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4882 - accuracy: 0.8422 - val_loss: 0.4723 - val_accuracy: 0.8481 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4643 - accuracy: 0.8558 - val_loss: 0.4740 - val_accuracy: 0.8491 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4406 - accuracy: 0.8667 - val_loss: 0.4313 - val_accuracy: 0.8682 - lr: 0.0010
Epoch 7/2000
383/383 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval()

0.895

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################

model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 5s 11ms/step - loss: 1.3102 - accuracy: 0.5779 - val_loss: 0.7070 - val_accuracy: 0.7507 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.6338 - accuracy: 0.7814 - val_loss: 0.5697 - val_accuracy: 0.8127 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.5559 - accuracy: 0.8154 - val_loss: 0.5158 - val_accuracy: 0.8337 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.5010 - accuracy: 0.8418 - val_loss: 0.4802 - val_accuracy: 0.8527 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4655 - accuracy: 0.8553 - val_loss: 0.4506 - val_accuracy: 0.8637 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 4s 10ms/step - loss: 0.4408 - accuracy: 0.8658 - val_loss: 0.4524 - val_accuracy: 0.8623 - lr: 0.0010
Epoch 7/2000
383/383 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')
model_eval()

0.8908

# 남은 데이터 라벨링하기


In [ ]:
model1 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model2 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [ ]:
unlab_label_1 = model1.predict(unlab_data_1)

In [ ]:
unlab_label_2 = model2.predict(unlab_data_2)

In [ ]:
train_data_1 = np.concatenate([train_data_1, unlab_data_1], axis=0)
train_labels_1 = np.concatenate([train_labels_1, unlab_label_1], axis=0)

In [ ]:
train_data_2 = np.concatenate([train_data_2, unlab_data_2], axis=0)
train_labels_2 = np.concatenate([train_labels_2, unlab_label_2], axis=0)

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
657/657 [==============================] - 8s 11ms/step - loss: 1.0349 - accuracy: 0.6429 - val_loss: 0.6362 - val_accuracy: 0.7989 - lr: 0.0010
Epoch 2/2000
657/657 [==============================] - 7s 10ms/step - loss: 0.5660 - accuracy: 0.8134 - val_loss: 0.5578 - val_accuracy: 0.8264 - lr: 0.0010
Epoch 3/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.5058 - accuracy: 0.8400 - val_loss: 0.4932 - val_accuracy: 0.8588 - lr: 0.0010
Epoch 4/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4663 - accuracy: 0.8589 - val_loss: 0.4715 - val_accuracy: 0.8762 - lr: 0.0010
Epoch 5/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4360 - accuracy: 0.8760 - val_loss: 0.4621 - val_accuracy: 0.8806 - lr: 0.0010
Epoch 6/2000
657/657 [==============================] - 7s 10ms/step - loss: 0.4188 - accuracy: 0.8819 - val_loss: 0.4362 - val_accuracy: 0.8884 - lr: 0.0010
Epoch 7/2000
657/657 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')
model_eval()

0.8952

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
657/657 [==============================] - 8s 11ms/step - loss: 1.0688 - accuracy: 0.6314 - val_loss: 0.6529 - val_accuracy: 0.7936 - lr: 0.0010
Epoch 2/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.5860 - accuracy: 0.8049 - val_loss: 0.5950 - val_accuracy: 0.8197 - lr: 0.0010
Epoch 3/2000
657/657 [==============================] - 7s 10ms/step - loss: 0.5214 - accuracy: 0.8291 - val_loss: 0.5220 - val_accuracy: 0.8512 - lr: 0.0010
Epoch 4/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4783 - accuracy: 0.8530 - val_loss: 0.4864 - val_accuracy: 0.8728 - lr: 0.0010
Epoch 5/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4484 - accuracy: 0.8699 - val_loss: 0.4764 - val_accuracy: 0.8792 - lr: 0.0010
Epoch 6/2000
657/657 [==============================] - 7s 11ms/step - loss: 0.4260 - accuracy: 0.8772 - val_loss: 0.4519 - val_accuracy: 0.8859 - lr: 0.0010
Epoch 7/2000
657/657 [==============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')
model_eval()

0.8944